In [532]:
# importing libraries
from itertools import product
import pandas as pd

# Static Experiments
## Comparing Runtimes of Algorithms

In [533]:
# opening static experiment
static_experiment_data = pd.read_csv('/Users/chloerainezeller/Desktop/Research/Color Experimental Data/Experiment 1 - Static Semantic Memory Algorithm/raw-data_COPY.csv')
static_experiment_data = static_experiment_data[static_experiment_data.num_episodes > 1]

# separating algorithmic data
bf_static = static_experiment_data[static_experiment_data.algorithm == 'brute-force']
exact_heuristic_static = static_experiment_data[static_experiment_data.algorithm == 'exact-heuristic']
neighbor_heuristic_static = static_experiment_data[static_experiment_data.algorithm == 'neighbor-heuristic']

In [534]:
num_episodes = static_experiment_data.num_episodes.unique()
num_labels = static_experiment_data.num_labels.unique()

FIXME: There are 3 extraneous columns in the data that need to be removed. Why these columns were added remains unclear.

### Brute Force Pivot Table and Analysis

In [535]:
bf_mean = bf_static.groupby(by = ['algorithm', 'num_episodes', 'num_labels']).mean()[['runtime', 'tpe']]
std_dev1 = bf_static.groupby(by=['algorithm', 'num_episodes', 'num_labels']).std()[['runtime', 'tpe']]
bf_std_dev = std_dev1.rename(columns={"runtime": "runtime_std_dev", "tpe": "tpe_std_dev"})

bf_pivot_table = pd.concat([bf_mean, bf_std_dev], axis=1)

### Exact Heuristic Pivot Table and Analysis

In [536]:
exact_heuristic_mean = exact_heuristic_static.groupby(by = ['algorithm', 'num_episodes', 'num_labels']).mean()[['runtime', 'tpe']]
std_dev2 = exact_heuristic_static.groupby(by = ['algorithm', 'num_episodes', 'num_labels']).std()[['runtime', 'tpe']]
exact_heuristic_std_dev = std_dev2.rename(columns = {'rutime': 'runtime_std_dev', 'tpe': 'tpe_std_dev'})                                                          
                                                            
exact_label_pivot_table = pd.concat([exact_heuristic_mean, exact_heuristic_std_dev], axis=1)

### Neighbor Label Pivot Table and Analysis
Note: this data was run with the original neighbor label algorithm, it should be re-run to make sure the bugs in the original algorithm didn't affect performance or results. 
#### FIXME

### Comparing Differing Returned Answers Between 3 Algorithms (BF, Exact Label and Neighbor Label)

In [537]:
bf_exact_heuristic_data = pd.concat([bf_static, exact_heuristic_static])

In [538]:
# define the parameters we care about
row_params = ['num_episodes','num_labels','random_seed', 'target_color']
col_params = ['algorithm', 'num_neighbors']
val_params = ['answer']


def string_pivot(df, row_params, col_params, val_params):
    # copy and extract the relevant columns
    answer_df = df.copy()[row_params + col_params + val_params]
    # reshaping magic - create an index using all the relelvant parameters
    # then unstack it to turn the column parameters in to columns
    answer_df = answer_df.set_index(row_params + col_params).unstack(level=col_params)
    # compute the new column names
    uniques = set(tuple(values) for values in df[col_params].copy().drop_duplicates().values.tolist())
    print(uniques)
    columns = []
    for column in product(*answer_df.columns.levels):
        print(column[1:])
        if column[1:] in uniques:
            print('add')
            columns.append([str(col) for col in column])
    # rename the columns
    answer_df.columns = ['_'.join([*values[1:], values[0]]) for values in columns]
    #answer_df.columns = ['_'.join([col, fn]) for fn, col in columns]
    # flatten the index
    answer_df = answer_df.reset_index()
    return answer_df

# copy and extract the relevant columns
answer_df = string_pivot(
    bf_exact_label_data[bf_exact_label_data.num_trials == 0],
    row_params,
    col_params,
    val_params)
answer_df.head()

{('exact-heuristic', 1), ('brute-force', 1)}
('brute-force', 1)
add
('exact-heuristic', 1)
add


,num_episodes,num_labels,random_seed,target_color,brute-force_1_answer,exact-heuristic_1_answer
0,1000,20,0.032582,#057D68,#187268,#187268
1,1000,20,0.032582,#06FDA3,#0FF9B5,#0FF9B5
2,1000,20,0.032582,#25AF13,#2DA317,#2DA317
3,1000,20,0.032582,#46E35F,#4AE451,#4AE451
4,1000,20,0.032582,#608EC6,#6096DC,#6096DC


In [539]:
diff_answer = answer_df[answer_df['brute-force_1_answer'] != answer_df['exact-heuristic_1_answer']]
same_answer = answer_df[answer_df['brute-force_1_answer'] == answer_df['exact-heuristic_1_answer']]

In [540]:
diff_answer.head()

,num_episodes,num_labels,random_seed,target_color,brute-force_1_answer,exact-heuristic_1_answer


### Different Answer Analysis

The answer differs between brute force and exact heuristic 8.96% of the time: 

In [541]:
num_different_answers = diff_answer.shape[0]
num_same_answers = same_answer.shape[0]

percent_diff_answer = num_different_answers / (num_different_answers + num_same_answers)
percent_diff_answer

0.0

In [542]:
# generate a list with the count of how many answers differed for each type of number of episodes
diff_episode_count = []
for number in num_episodes:
    diff_episode_count.append(diff_answer[diff_answer.num_episodes == number].shape[0])

In [543]:
diff_num_episode_count = list(zip(num_episodes, episode_count))
diff_num_episode_count

[(10, 20), (100, 16), (1000, 10), (10000, 5), (100000, 5)]

As the number of episodes increases, there is a lower incidence of a difference in the asnwer returned. FIXME: WHY?

In [544]:
diff_label_count = []
for number in num_labels:
    diff_label_count.append(diff_answer[diff_answer.num_labels == number].shape[0])

In [545]:
num_label_count = list(zip(num_labels, label_count))
num_label_count

[(5, 19), (20, 11), (45, 8), (70, 5), (150, 13)]

As the number of labels increases, the number of different answers decreases until 150 labels, where it spikes again. 
FIXME: WHY?

### Same Answer Analysis

The answer is the same between brute force and exact heuristic 91.04% of the time:

In [546]:
percent_same_answer = 1 - percent_diff_answer
percent_same_answer

1.0

In [547]:
same_episode_count = []
for number in num_episodes:
    same_episode_count.append(same_answer[same_answer.num_episodes == number].shape[0])

In [548]:
same_num_episode_count = list(zip(num_episodes, same_episode_count))
same_num_episode_count

[(10, 0), (100, 0), (1000, 300), (10000, 291), (100000, 0)]

In [549]:
same_label_count = []
for number in num_episodes:
    same_label_count.append(same_answer[same_answer.num_labels == number].shape[0])

In [550]:
same_num_label_count = list(zip(num_episodes, same_episode_count))
same_num_episode_count

[(10, 0), (100, 0), (1000, 300), (10000, 291), (100000, 0)]

----------------------------------------------------------------------------------------------------------------------

# Dynamic Experiments
## Brute Force, Exact Label and Neighbor Heuristic Algorithms

In [551]:
# Opening Dynamic Experiments 1 and 3
neighbor_label = pd.read_csv('/Users/chloerainezeller/Desktop/Research/Color Experimental Data/Experiment 2 - Dynamic Changing Semantic Labels/ChangingSemanticLabel4_NeighborLabel_RevisedAlgorithm.csv')
brute_force_exact_label = pd.read_csv('/Users/chloerainezeller/Desktop/Research/Color Experimental Data/Experiment 2 - Dynamic Changing Semantic Labels/Changing_Semantic_Label.csv')

#fallbacks = neighbor_label[neighbor_label.num_fallbacks == 1]
neighbor_label = neighbor_label[(neighbor_label.num_labels > 10)]
fallbacks = neighbor_label.copy()[neighbor_label.num_labels > 10]
brute_force = brute_force_exact_label[(brute_force_exact_label.num_labels > 10) & (brute_force_exact_label.algorithm == 'brute-force')]
exact_label = brute_force_exact_label[(brute_force_exact_label.num_labels > 10) & (brute_force_exact_label.algorithm == 'exact-heuristic')]

In [552]:
# Checked the results of these, both are correct
columns = ['algorithm', 'num_episodes', 'num_labels', 'num_trials', 'num_neighbors',
           'random_seed', 'target_color', 'target_color_hex', 'color_sequence_type',
           '_hostname', '_pwd', '_username', '_start_time', '_end_time', 'answer', 
           'num_fallbacks', 'runtime', 'total_episodes']
filtered_bf = brute_force.filter(items = columns)
filtered_exact_label = exact_label.filter(items = columns)
filtered_fallbacks = fallbacks.filter(items = columns)


In [553]:
# first attempt at trying to merge together the fallback data with the same answers as brute force
bf_exact_label_data = pd.concat([brute_force, exact_label])
bf_neighbor_label_data = pd.concat([brute_force, exact_label, neighbor_label])

In [554]:
num_episodes = bf_exact_label_data.num_episodes.unique()
num_labels = bf_exact_label_data.num_labels.unique()

## Generate Answer DataFrame to Compare BF and Exact Label

In [555]:
# define the parameters we care about
row_params = ['num_episodes','num_labels','random_seed', 'target_color']
col_params = ['algorithm', 'num_neighbors']
val_params = ['answer', 'num_fallbacks']

# copy and extract the relevant columns
answer_df = string_pivot(
    bf_neighbor_label_data[bf_neighbor_label_data.num_trials == 0],
    row_params,
    col_params,
    val_params)

# write answer data frame to a csv file
#answer_df.to_csv('answer_comparison.csv', sep=',')

# add a column to compare the answers from the two algorithms
#answer_df['same'] = (answer_df['brute-force_answer'] == answer_df['exact-heuristic_answer'])
 
# count the different values in that column
#answer_df['same'].value_counts()
answer_df

{('neighbor-heuristic', 3), ('exact-heuristic', 1), ('brute-force', 1), ('neighbor-heuristic', 1), ('neighbor-heuristic', 2)}
('brute-force', 1)
add
('brute-force', 2)
('brute-force', 3)
('exact-heuristic', 1)
add
('exact-heuristic', 2)
('exact-heuristic', 3)
('neighbor-heuristic', 1)
add
('neighbor-heuristic', 2)
add
('neighbor-heuristic', 3)
add
('brute-force', 1)
add
('brute-force', 2)
('brute-force', 3)
('exact-heuristic', 1)
add
('exact-heuristic', 2)
('exact-heuristic', 3)
('neighbor-heuristic', 1)
add
('neighbor-heuristic', 2)
add
('neighbor-heuristic', 3)
add


,num_episodes,num_labels,random_seed,target_color,brute-force_1_answer,exact-heuristic_1_answer,neighbor-heuristic_1_answer,neighbor-heuristic_2_answer,neighbor-heuristic_3_answer,brute-force_1_num_fallbacks,exact-heuristic_1_num_fallbacks,neighbor-heuristic_1_num_fallbacks,neighbor-heuristic_2_num_fallbacks,neighbor-heuristic_3_num_fallbacks
0,1000,20,0.032582,#057D68,#187268,#187268,#029770,#029770,#029770,0.0,1.0,0.0,0.0,0.0
1,1000,20,0.032582,#06FDA3,#0FF9B5,#0FF9B5,#0CD9B9,#0CD9B9,#0CD9B9,0.0,1.0,0.0,0.0,0.0
2,1000,20,0.032582,#25AF13,#2DA317,#2DA317,#2DA317,#2DA317,#2DA317,0.0,1.0,0.0,0.0,0.0
3,1000,20,0.032582,#46E35F,#4AE451,#4AE451,#4DF169,#4DF169,#4DF169,0.0,1.0,0.0,0.0,0.0
4,1000,20,0.032582,#608EC6,#6096DC,#6096DC,#6096DC,#6096DC,#6096DC,0.0,1.0,0.0,0.0,0.0
5,1000,20,0.032582,#74B32E,#83B12C,#83B12C,#7DCD3E,#7DCD3E,#7DCD3E,0.0,1.0,0.0,0.0,0.0
6,1000,20,0.032582,#757988,#749282,#749282,#676792,#676792,#676792,0.0,1.0,0.0,0.0,0.0
7,1000,20,0.032582,#BCB249,#B1B74F,#B1B74F,#BFB15B,#BFB15B,#BFB15B,0.0,1.0,0.0,0.0,0.0
8,1000,20,0.032582,#CD6D8F,#C76D9B,#C76D9B,#BF6991,#BF6991,#BF6991,0.0,1.0,0.0,0.0,0.0
9,1000,20,0.032582,#F71976,#E7167D,#E7167D,#FE1089,#FE1089,#FE1089,0.0,1.0,0.0,0.0,0.0


In [556]:
diff_answer = answer_df[answer_df['brute-force_1_answer'] != answer_df['exact-heuristic_1_answer']]
same_answer = answer_df[answer_df['brute-force_1_answer'] == answer_df['exact-heuristic_1_answer']]
same_answer

,num_episodes,num_labels,random_seed,target_color,brute-force_1_answer,exact-heuristic_1_answer,neighbor-heuristic_1_answer,neighbor-heuristic_2_answer,neighbor-heuristic_3_answer,brute-force_1_num_fallbacks,exact-heuristic_1_num_fallbacks,neighbor-heuristic_1_num_fallbacks,neighbor-heuristic_2_num_fallbacks,neighbor-heuristic_3_num_fallbacks
0,1000,20,0.032582,#057D68,#187268,#187268,#029770,#029770,#029770,0.0,1.0,0.0,0.0,0.0
1,1000,20,0.032582,#06FDA3,#0FF9B5,#0FF9B5,#0CD9B9,#0CD9B9,#0CD9B9,0.0,1.0,0.0,0.0,0.0
2,1000,20,0.032582,#25AF13,#2DA317,#2DA317,#2DA317,#2DA317,#2DA317,0.0,1.0,0.0,0.0,0.0
3,1000,20,0.032582,#46E35F,#4AE451,#4AE451,#4DF169,#4DF169,#4DF169,0.0,1.0,0.0,0.0,0.0
4,1000,20,0.032582,#608EC6,#6096DC,#6096DC,#6096DC,#6096DC,#6096DC,0.0,1.0,0.0,0.0,0.0
5,1000,20,0.032582,#74B32E,#83B12C,#83B12C,#7DCD3E,#7DCD3E,#7DCD3E,0.0,1.0,0.0,0.0,0.0
6,1000,20,0.032582,#757988,#749282,#749282,#676792,#676792,#676792,0.0,1.0,0.0,0.0,0.0
7,1000,20,0.032582,#BCB249,#B1B74F,#B1B74F,#BFB15B,#BFB15B,#BFB15B,0.0,1.0,0.0,0.0,0.0
8,1000,20,0.032582,#CD6D8F,#C76D9B,#C76D9B,#BF6991,#BF6991,#BF6991,0.0,1.0,0.0,0.0,0.0
9,1000,20,0.032582,#F71976,#E7167D,#E7167D,#FE1089,#FE1089,#FE1089,0.0,1.0,0.0,0.0,0.0


### Different Answer Analysis

In [557]:
num_diff_answers = diff_answer.shape[0]
num_same_answers = diff_answer.shape[0]

#percent_diff_answer = num_diff_answers / (num_diff_answers + num_same_answers)


In [558]:
diff_episode_count = []
for number in num_episodes:
    diff_episode_count.append(diff_answer[diff_answer.num_episodes == number].shape[0])

In [559]:
diff_num_episode_count = list(zip(num_episodes, diff_episode_count))
diff_num_episode_count

[(1000, 0), (10000, 9)]

As the number of episodes increases, the incidence of differing answers returned by the BF and exact label algorithms decreases. 

In [560]:
diff_label_count = []
for number in num_labels:
    diff_label_count.append(diff_answer[diff_answer.num_labels == number].shape[0])

In [561]:
diff_num_label_count = list(zip(num_labels, diff_label_count))
diff_num_label_count

[(20, 0), (50, 0), (100, 9)]

As the number of labels increases, the incidence of differing answers retuned by the BF and exact label algorithms decreases until we reach 150 labels, at ehich point it increases once again. 